### ผลการทดลองที่ 2

1. เขียนโปรแกรมเพื่อเพิ่มข้อมูลนักศึกษา 5 คน รันโปรแกรมและบันทึกรูปผลการรันโปรแกรม

In [7]:
class BTreeNode:
    def __init__(self, leaf=True):
        self.leaf = leaf  # เป็นใบหรือไม่
        self.keys = []  # เก็บค่าของโหนด (เรียงจากน้อยไปมาก)
        self.data = []  # ข้อมูลที่สัมพันธ์กับ key
        self.children = []  # ลูกของโหนด (ถ้าไม่ใช่ใบ)

class BTree:
    def __init__(self, order):
        self.root = None  # โหนดราก
        self.order = order  # ระดับของ B-Tree

    def get_max_keys(self):
        return self.order - 1  # จำนวน key สูงสุดในโหนด

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode()
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.get_max_keys():
            new_root = BTreeNode(leaf=False)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.get_max_keys():
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf)
        mid = (order - 1) // 2  # คำนวณค่ากลางที่ถูกต้อง

        # ดึงค่า mid ขึ้นไป parent
        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        # ก๊อบปี้ค่าที่เหลือไปยังโหนดใหม่
        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        # ตัดค่าที่เคลื่อนย้ายออกจากโหนดเดิม
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def traverse(self, node=None, level=0):
        if node is None:
            node = self.root
        print("Level", level, "Keys:", node.keys)
        if not node.leaf:
            for child in node.children:
                self.traverse(child, level + 1)

# เพิ่มข้อมูลนักศึกษา
btree = BTree(3)
students = [(192, "สุภัทรา"), (130, "พีรพัฒน์"), (270, "ภูมิ"), (262, "พีรวิชญ์"), (145, "มณฑล")]

for student_id, name in students:
    btree.insert(student_id, name)

# แสดงโครงสร้าง B-Tree
btree.traverse()



Level 0 Keys: [192]
Level 1 Keys: [130, 145]
Level 1 Keys: [262, 270]


2. แก้ไข class B-Tree ให้มีการเก็บจำนวน Entry สูงสุด และต่ำสุด แทนการใช้ get_min_keys และ get_max_keys

In [8]:
class BTreeNode:
    def __init__(self, leaf=True, min_keys=None, max_keys=None):
        self.leaf = leaf  # เป็นใบหรือไม่
        self.keys = []  # เก็บค่าของโหนด (เรียงจากน้อยไปมาก)
        self.data = []  # ข้อมูลที่สัมพันธ์กับ key
        self.children = []  # ลูกของโหนด (ถ้าไม่ใช่ใบ)
        self.min_keys = min_keys  # จำนวน key ต่ำสุดในโหนด
        self.max_keys = max_keys  # จำนวน key สูงสุดในโหนด

class BTree:
    def __init__(self, order):
        self.root = None  # โหนดราก
        self.order = order  # ระดับของ B-Tree
        self.min_keys = (order - 1) // 2  # จำนวน key ต่ำสุด
        self.max_keys = order - 1  # จำนวน key สูงสุดในโหนด

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode(min_keys=self.min_keys, max_keys=self.max_keys)
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False, min_keys=self.min_keys, max_keys=self.max_keys)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf, min_keys=self.min_keys, max_keys=self.max_keys)
        mid = (order - 1) // 2  # คำนวณค่ากลางที่ถูกต้อง

        # ดึงค่า mid ขึ้นไป parent
        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        # ก๊อบปี้ค่าที่เหลือไปยังโหนดใหม่
        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        # ตัดค่าที่เคลื่อนย้ายออกจากโหนดเดิม
        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def traverse(self, node=None, level=0):
        if node is None:
            node = self.root
        print("Level", level, "Keys:", node.keys)
        if not node.leaf:
            for child in node.children:
                self.traverse(child, level + 1)

# เพิ่มข้อมูลนักศึกษา
btree = BTree(3)
students = [(192, "สุภัทรา"), (130, "พีรพัฒน์"), (270, "ภูมิ"), (262, "พีรวิชญ์"), (145, "มณฑล")]

for student_id, name in students:
    btree.insert(student_id, name)

# แสดงโครงสร้าง B-Tree
btree.traverse()


Level 0 Keys: [192]
Level 1 Keys: [130, 145]
Level 1 Keys: [262, 270]


### ผลการทดลองที่ 3

1. เขียนโปรแกรมเพื่อทดสอบการค้นหาข้อมูลใน B-Tree ตามข้อมูลที่ได้เพิ่มในการทดลองก่อนหน้า

In [16]:
class BTreeNode:
    def __init__(self, leaf=True, min_keys=None, max_keys=None):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []
        self.min_keys = min_keys
        self.max_keys = max_keys

class BTree:
    def __init__(self, order):
        self.root = None
        self.order = order
        self.min_keys = (order - 1) // 2
        self.max_keys = order - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode(min_keys=self.min_keys, max_keys=self.max_keys)
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False, min_keys=self.min_keys, max_keys=self.max_keys)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf, min_keys=self.min_keys, max_keys=self.max_keys)
        mid = (order - 1) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def search(self, key):
        """ค้นหาข้อมูลจาก key ที่กำหนด"""
        def _search_node(node, key):
            i = 0
            # หาตำแหน่งที่เหมาะสม
            while i < len(node.keys) and key > node.keys[i]:
                i += 1

            # ถ้าเจอ key ที่ตรงกัน
            if i < len(node.keys) and key == node.keys[i]:
                return (node, i)

            # ถ้าเป็นใบและไม่เจอ key
            if node.leaf:
                return None

            # ค้นหาในลูกที่เหมาะสม
            return _search_node(node.children[i], key)

        if self.root is None:
            return None

        result = _search_node(self.root, key)
        if result:
            node, index = result
            return node.data[index]
        return None

    def traverse(self, node=None, level=0):
        if node is None:
            node = self.root
        print("Level", level, "Keys:", node.keys)
        if not node.leaf:
            for child in node.children:
                self.traverse(child, level + 1)


# สร้าง B-Tree และเพิ่มข้อมูลนักศึกษา
btree = BTree(3)
students = [(192, "สุภัทรา"), (130, "พีรพัฒน์"), (270, "ภูมิ"), (262, "พีรวิชญ์"), (145, "มณฑล")]

for student_id, name in students:
    btree.insert(student_id, name)

# ทดสอบการค้นหาข้อมูล
search_results = [
    btree.search(192),  # ค้นหาข้อมูลนักศึกษาที่มีรหัส 192
    btree.search(130),  # ค้นหาข้อมูลนักศึกษาที่มีรหัส 130
    btree.search(270),  # ค้นหาข้อมูลนักศึกษาที่มีรหัส 270
    btree.search(262),  # ค้นหาข้อมูลนักศึกษาที่มีรหัส 262
    btree.search(145),  # ค้นหาข้อมูลนักศึกษาที่มีรหัส 145
    btree.search(100)   # ค้นหาข้อมูลที่ไม่มีอยู่
]

# แสดงผลลัพธ์ของการค้นหาทั้งหมด
search_results

['สุภัทรา', 'พีรพัฒน์', 'ภูมิ', 'พีรวิชญ์', 'มณฑล', None]

### ผลการทดลองที่ 4

1. เขียนโปรแกรมเพื่อทดสอบการแสดงข้อมูลใน B-Tree 

In [10]:
class BTreeNode:
    def __init__(self, leaf=True, min_keys=None, max_keys=None):
        self.leaf = leaf
        self.keys = []
        self.data = []
        self.children = []
        self.min_keys = min_keys
        self.max_keys = max_keys

class BTree:
    def __init__(self, order):
        self.root = None
        self.order = order
        self.min_keys = (order - 1) // 2
        self.max_keys = order - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode(min_keys=self.min_keys, max_keys=self.max_keys)
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False, min_keys=self.min_keys, max_keys=self.max_keys)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf, min_keys=self.min_keys, max_keys=self.max_keys)
        mid = (order - 1) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def display(self):
        """แสดงโครงสร้างของ B-Tree"""
        def _display(node, level):
            if node:
                # แสดง key และ data ในโหนดปัจจุบัน
                print('  ' * level + f"Keys: {node.keys}")
                print('  ' * level + f"Data: {node.data}")
                print('  ' * level + f"Is Leaf: {node.leaf}")
                print('  ' * level + f"Number of children: {len(node.children)}")
                print()
                # แสดงลูกทุกตัวแบบ recursive
                for child in node.children:
                    _display(child, level + 1)
        
        print("B-Tree Structure:")
        _display(self.root, 0)

# สร้าง B-Tree และเพิ่มข้อมูลนักศึกษา
btree = BTree(3)
students = [(192, "สุภัทรา"), (130, "พีรพัฒน์"), (270, "ภูมิ"), (262, "พีรวิชญ์"), (145, "มณฑล")]

for student_id, name in students:
    btree.insert(student_id, name)

# ทดสอบการแสดงผล B-Tree
btree.display()


B-Tree Structure:
Keys: [192]
Data: ['สุภัทรา']
Is Leaf: False
Number of children: 2

  Keys: [130, 145]
  Data: ['พีรพัฒน์', 'มณฑล']
  Is Leaf: True
  Number of children: 0

  Keys: [262, 270]
  Data: ['พีรวิชญ์', 'ภูมิ']
  Is Leaf: True
  Number of children: 0



### แบบฝึกหัด

1. ให้นักศึกษาเพิ่มเมธอดสำหรับลบข้อมูล (key และ data) ออกจาก B-Tree

In [11]:
class BTree:
    def __init__(self, order):
        self.root = None
        self.order = order
        self.min_keys = (order - 1) // 2
        self.max_keys = order - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode(min_keys=self.min_keys, max_keys=self.max_keys)
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False, min_keys=self.min_keys, max_keys=self.max_keys)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf, min_keys=self.min_keys, max_keys=self.max_keys)
        mid = (order - 1) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def display(self):
        """แสดงโครงสร้างของ B-Tree"""
        def _display(node, level):
            if node:
                # แสดง key และ data ในโหนดปัจจุบัน
                print('  ' * level + f"Keys: {node.keys}")
                print('  ' * level + f"Data: {node.data}")
                print('  ' * level + f"Is Leaf: {node.leaf}")
                print('  ' * level + f"Number of children: {len(node.children)}")
                print()
                # แสดงลูกทุกตัวแบบ recursive
                for child in node.children:
                    _display(child, level + 1)
        
        print("B-Tree Structure:")
        _display(self.root, 0)

    def delete(self, key):
        """ลบข้อมูลจาก B-Tree"""
        if self.root is None:
            print("The tree is empty!")
            return

        self._delete(self.root, key)

        # ถ้า root มีแค่ 1 ลูกหลังจากการลบ, ทำให้ root เป็นลูก
        if len(self.root.keys) == 0:
            if not self.root.leaf:
                self.root = self.root.children[0]
            else:
                self.root = None

    def _delete(self, node, key):
        """ฟังก์ชันลบในโหนด"""
        i = 0
        while i < len(node.keys) and key > node.keys[i]:
            i += 1

        if i < len(node.keys) and key == node.keys[i]:
            # ถ้า key อยู่ในโหนดใบ
            if node.leaf:
                node.keys.pop(i)
                node.data.pop(i)
            else:
                # ถ้า key อยู่ในโหนดที่ไม่ใช่ใบ
                self._delete_internal_node(node, i)
        elif not node.leaf:
            self._delete_in_non_leaf(node, i, key)

    def _delete_internal_node(self, node, i):
        """จัดการการลบจากโหนดที่ไม่ใช่ใบ"""
        key = node.keys[i]
        left_child = node.children[i]
        right_child = node.children[i + 1]

        if len(left_child.keys) > self.min_keys:
            predecessor = self._get_predecessor(left_child)
            node.keys[i] = predecessor
            node.data[i] = predecessor[1]
            self._delete(left_child, predecessor[0])
        elif len(right_child.keys) > self.min_keys:
            successor = self._get_successor(right_child)
            node.keys[i] = successor
            node.data[i] = successor[1]
            self._delete(right_child, successor[0])
        else:
            self._merge_children(node, i)
            self._delete(left_child, key)

    def _get_predecessor(self, node):
        """ค้นหาค่าที่มากที่สุดในลูกซ้าย"""
        while not node.leaf:
            node = node.children[-1]
        return node.keys[-1], node.data[-1]

    def _get_successor(self, node):
        """ค้นหาค่าที่น้อยที่สุดในลูกขวา"""
        while not node.leaf:
            node = node.children[0]
        return node.keys[0], node.data[0]

    def _merge_children(self, node, i):
        """รวมลูกทั้งสองให้เป็นหนึ่ง"""
        left_child = node.children[i]
        right_child = node.children[i + 1]

        left_child.keys.append(node.keys[i])
        left_child.data.append(node.data[i])

        left_child.keys.extend(right_child.keys)
        left_child.data.extend(right_child.data)

        if not left_child.leaf:
            left_child.children.extend(right_child.children)

        node.keys.pop(i)
        node.data.pop(i)
        node.children.pop(i + 1)

    def _delete_in_non_leaf(self, node, i, key):
        """จัดการการลบในลูกที่ไม่ใช่ใบ"""
        child = node.children[i]

        if len(child.keys) > self.min_keys:
            self._delete(child, key)
        else:
            left_sibling = None
            right_sibling = None

            if i > 0:
                left_sibling = node.children[i - 1]
            if i < len(node.children) - 1:
                right_sibling = node.children[i + 1]

            if left_sibling and len(left_sibling.keys) > self.min_keys:
                self._borrow_from_left(node, i)
            elif right_sibling and len(right_sibling.keys) > self.min_keys:
                self._borrow_from_right(node, i)
            else:
                if left_sibling:
                    self._merge_children(node, i - 1)
                else:
                    self._merge_children(node, i)

                self._delete(child, key)

    def _borrow_from_left(self, node, i):
        """ยืมคีย์จากลูกซ้าย"""
        child = node.children[i]
        left_sibling = node.children[i - 1]

        child.keys.insert(0, node.keys[i - 1])
        child.data.insert(0, node.data[i - 1])

        node.keys[i - 1] = left_sibling.keys.pop()
        node.data[i - 1] = left_sibling.data.pop()

        if not left_sibling.leaf:
            child.children.insert(0, left_sibling.children.pop())

    def _borrow_from_right(self, node, i):
        """ยืมคีย์จากลูกขวา"""
        child = node.children[i]
        right_sibling = node.children[i + 1]

        child.keys.append(node.keys[i])
        child.data.append(node.data[i])

        node.keys[i] = right_sibling.keys.pop(0)
        node.data[i] = right_sibling.data.pop(0)

        if not right_sibling.leaf:
            child.children.append(right_sibling.children.pop(0))


2. ให้นักศึกษาเพิ่มเมธอดสำหรับอัปเดตข้อมูล (data) สำหรับ key ที่กำหนด

In [12]:
class BTree:
    def __init__(self, order):
        self.root = None
        self.order = order
        self.min_keys = (order - 1) // 2
        self.max_keys = order - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode(min_keys=self.min_keys, max_keys=self.max_keys)
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False, min_keys=self.min_keys, max_keys=self.max_keys)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf, min_keys=self.min_keys, max_keys=self.max_keys)
        mid = (order - 1) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def display(self):
        """แสดงโครงสร้างของ B-Tree"""
        def _display(node, level):
            if node:
                # แสดง key และ data ในโหนดปัจจุบัน
                print('  ' * level + f"Keys: {node.keys}")
                print('  ' * level + f"Data: {node.data}")
                print('  ' * level + f"Is Leaf: {node.leaf}")
                print('  ' * level + f"Number of children: {len(node.children)}")
                print()
                # แสดงลูกทุกตัวแบบ recursive
                for child in node.children:
                    _display(child, level + 1)
        
        print("B-Tree Structure:")
        _display(self.root, 0)

    def delete(self, key):
        """ลบข้อมูลจาก B-Tree"""
        if self.root is None:
            print("The tree is empty!")
            return

        self._delete(self.root, key)

        # ถ้า root มีแค่ 1 ลูกหลังจากการลบ, ทำให้ root เป็นลูก
        if len(self.root.keys) == 0:
            if not self.root.leaf:
                self.root = self.root.children[0]
            else:
                self.root = None

    def _delete(self, node, key):
        """ฟังก์ชันลบในโหนด"""
        i = 0
        while i < len(node.keys) and key > node.keys[i]:
            i += 1

        if i < len(node.keys) and key == node.keys[i]:
            # ถ้า key อยู่ในโหนดใบ
            if node.leaf:
                node.keys.pop(i)
                node.data.pop(i)
            else:
                # ถ้า key อยู่ในโหนดที่ไม่ใช่ใบ
                self._delete_internal_node(node, i)
        elif not node.leaf:
            self._delete_in_non_leaf(node, i, key)

    def _delete_internal_node(self, node, i):
        """จัดการการลบจากโหนดที่ไม่ใช่ใบ"""
        key = node.keys[i]
        left_child = node.children[i]
        right_child = node.children[i + 1]

        if len(left_child.keys) > self.min_keys:
            predecessor = self._get_predecessor(left_child)
            node.keys[i] = predecessor
            node.data[i] = predecessor[1]
            self._delete(left_child, predecessor[0])
        elif len(right_child.keys) > self.min_keys:
            successor = self._get_successor(right_child)
            node.keys[i] = successor
            node.data[i] = successor[1]
            self._delete(right_child, successor[0])
        else:
            self._merge_children(node, i)
            self._delete(left_child, key)

    def _get_predecessor(self, node):
        """ค้นหาค่าที่มากที่สุดในลูกซ้าย"""
        while not node.leaf:
            node = node.children[-1]
        return node.keys[-1], node.data[-1]

    def _get_successor(self, node):
        """ค้นหาค่าที่น้อยที่สุดในลูกขวา"""
        while not node.leaf:
            node = node.children[0]
        return node.keys[0], node.data[0]

    def _merge_children(self, node, i):
        """รวมลูกทั้งสองให้เป็นหนึ่ง"""
        left_child = node.children[i]
        right_child = node.children[i + 1]

        left_child.keys.append(node.keys[i])
        left_child.data.append(node.data[i])

        left_child.keys.extend(right_child.keys)
        left_child.data.extend(right_child.data)

        if not left_child.leaf:
            left_child.children.extend(right_child.children)

        node.keys.pop(i)
        node.data.pop(i)
        node.children.pop(i + 1)

    def _delete_in_non_leaf(self, node, i, key):
        """จัดการการลบในลูกที่ไม่ใช่ใบ"""
        child = node.children[i]

        if len(child.keys) > self.min_keys:
            self._delete(child, key)
        else:
            left_sibling = None
            right_sibling = None

            if i > 0:
                left_sibling = node.children[i - 1]
            if i < len(node.children) - 1:
                right_sibling = node.children[i + 1]

            if left_sibling and len(left_sibling.keys) > self.min_keys:
                self._borrow_from_left(node, i)
            elif right_sibling and len(right_sibling.keys) > self.min_keys:
                self._borrow_from_right(node, i)
            else:
                if left_sibling:
                    self._merge_children(node, i - 1)
                else:
                    self._merge_children(node, i)

                self._delete(child, key)

    def _borrow_from_left(self, node, i):
        """ยืมคีย์จากลูกซ้าย"""
        child = node.children[i]
        left_sibling = node.children[i - 1]

        child.keys.insert(0, node.keys[i - 1])
        child.data.insert(0, node.data[i - 1])

        node.keys[i - 1] = left_sibling.keys.pop()
        node.data[i - 1] = left_sibling.data.pop()

        if not left_sibling.leaf:
            child.children.insert(0, left_sibling.children.pop())

    def _borrow_from_right(self, node, i):
        """ยืมคีย์จากลูกขวา"""
        child = node.children[i]
        right_sibling = node.children[i + 1]

        child.keys.append(node.keys[i])
        child.data.append(node.data[i])

        node.keys[i] = right_sibling.keys.pop(0)
        node.data[i] = right_sibling.data.pop(0)

        if not right_sibling.leaf:
            child.children.append(right_sibling.children.pop(0))

    def update(self, key, new_data):
        """อัปเดตข้อมูลใน B-Tree"""
        if self.root is None:
            print("The tree is empty!")
            return

        node, index = self._search_for_update(self.root, key)

        if node is None:
            print(f"Key {key} not found in the tree!")
            return
        
        # อัปเดตข้อมูลที่พบ
        node.data[index] = new_data
        print(f"Updated key {key} with new data: {new_data}")

    def _search_for_update(self, node, key):
        """ค้นหาคีย์ที่ต้องการอัปเดต"""
        i = 0
        while i < len(node.keys) and key > node.keys[i]:
            i += 1

        if i < len(node.keys) and key == node.keys[i]:
            return node, i  # คืนค่า node และ index ของ key

        if node.leaf:
            return None, None

        return self._search_for_update(node.children[i], key)


3. ให้นักศึกษาเพิ่มเมธอดสำหรับแสดงข้อมูลทั้งหมดใน B-Tree เรียงตาม key

In [13]:
class BTree:
    def __init__(self, order):
        self.root = None
        self.order = order
        self.min_keys = (order - 1) // 2
        self.max_keys = order - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode(min_keys=self.min_keys, max_keys=self.max_keys)
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False, min_keys=self.min_keys, max_keys=self.max_keys)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf, min_keys=self.min_keys, max_keys=self.max_keys)
        mid = (order - 1) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def display(self):
        """แสดงโครงสร้างของ B-Tree"""
        def _display(node, level):
            if node:
                # แสดง key และ data ในโหนดปัจจุบัน
                print('  ' * level + f"Keys: {node.keys}")
                print('  ' * level + f"Data: {node.data}")
                print('  ' * level + f"Is Leaf: {node.leaf}")
                print('  ' * level + f"Number of children: {len(node.children)}")
                print()
                # แสดงลูกทุกตัวแบบ recursive
                for child in node.children:
                    _display(child, level + 1)
        
        print("B-Tree Structure:")
        _display(self.root, 0)

    def delete(self, key):
        """ลบข้อมูลจาก B-Tree"""
        if self.root is None:
            print("The tree is empty!")
            return

        self._delete(self.root, key)

        # ถ้า root มีแค่ 1 ลูกหลังจากการลบ, ทำให้ root เป็นลูก
        if len(self.root.keys) == 0:
            if not self.root.leaf:
                self.root = self.root.children[0]
            else:
                self.root = None

    def _delete(self, node, key):
        """ฟังก์ชันลบในโหนด"""
        i = 0
        while i < len(node.keys) and key > node.keys[i]:
            i += 1

        if i < len(node.keys) and key == node.keys[i]:
            # ถ้า key อยู่ในโหนดใบ
            if node.leaf:
                node.keys.pop(i)
                node.data.pop(i)
            else:
                # ถ้า key อยู่ในโหนดที่ไม่ใช่ใบ
                self._delete_internal_node(node, i)
        elif not node.leaf:
            self._delete_in_non_leaf(node, i, key)

    def _delete_internal_node(self, node, i):
        """จัดการการลบจากโหนดที่ไม่ใช่ใบ"""
        key = node.keys[i]
        left_child = node.children[i]
        right_child = node.children[i + 1]

        if len(left_child.keys) > self.min_keys:
            predecessor = self._get_predecessor(left_child)
            node.keys[i] = predecessor
            node.data[i] = predecessor[1]
            self._delete(left_child, predecessor[0])
        elif len(right_child.keys) > self.min_keys:
            successor = self._get_successor(right_child)
            node.keys[i] = successor
            node.data[i] = successor[1]
            self._delete(right_child, successor[0])
        else:
            self._merge_children(node, i)
            self._delete(left_child, key)

    def _get_predecessor(self, node):
        """ค้นหาค่าที่มากที่สุดในลูกซ้าย"""
        while not node.leaf:
            node = node.children[-1]
        return node.keys[-1], node.data[-1]

    def _get_successor(self, node):
        """ค้นหาค่าที่น้อยที่สุดในลูกขวา"""
        while not node.leaf:
            node = node.children[0]
        return node.keys[0], node.data[0]

    def _merge_children(self, node, i):
        """รวมลูกทั้งสองให้เป็นหนึ่ง"""
        left_child = node.children[i]
        right_child = node.children[i + 1]

        left_child.keys.append(node.keys[i])
        left_child.data.append(node.data[i])

        left_child.keys.extend(right_child.keys)
        left_child.data.extend(right_child.data)

        if not left_child.leaf:
            left_child.children.extend(right_child.children)

        node.keys.pop(i)
        node.data.pop(i)
        node.children.pop(i + 1)

    def _delete_in_non_leaf(self, node, i, key):
        """จัดการการลบในลูกที่ไม่ใช่ใบ"""
        child = node.children[i]

        if len(child.keys) > self.min_keys:
            self._delete(child, key)
        else:
            left_sibling = None
            right_sibling = None

            if i > 0:
                left_sibling = node.children[i - 1]
            if i < len(node.children) - 1:
                right_sibling = node.children[i + 1]

            if left_sibling and len(left_sibling.keys) > self.min_keys:
                self._borrow_from_left(node, i)
            elif right_sibling and len(right_sibling.keys) > self.min_keys:
                self._borrow_from_right(node, i)
            else:
                if left_sibling:
                    self._merge_children(node, i - 1)
                else:
                    self._merge_children(node, i)

                self._delete(child, key)

    def _borrow_from_left(self, node, i):
        """ยืมคีย์จากลูกซ้าย"""
        child = node.children[i]
        left_sibling = node.children[i - 1]

        child.keys.insert(0, node.keys[i - 1])
        child.data.insert(0, node.data[i - 1])

        node.keys[i - 1] = left_sibling.keys.pop()
        node.data[i - 1] = left_sibling.data.pop()

        if not left_sibling.leaf:
            child.children.insert(0, left_sibling.children.pop())

    def _borrow_from_right(self, node, i):
        """ยืมคีย์จากลูกขวา"""
        child = node.children[i]
        right_sibling = node.children[i + 1]

        child.keys.append(node.keys[i])
        child.data.append(node.data[i])

        node.keys[i] = right_sibling.keys.pop(0)
        node.data[i] = right_sibling.data.pop(0)

        if not right_sibling.leaf:
            child.children.append(right_sibling.children.pop(0))

    def update(self, key, new_data):
        """อัปเดตข้อมูลใน B-Tree"""
        if self.root is None:
            print("The tree is empty!")
            return

        node, index = self._search_for_update(self.root, key)

        if node is None:
            print(f"Key {key} not found in the tree!")
            return
        
        # อัปเดตข้อมูลที่พบ
        node.data[index] = new_data
        print(f"Updated key {key} with new data: {new_data}")

    def _search_for_update(self, node, key):
        """ค้นหาคีย์ที่ต้องการอัปเดต"""
        i = 0
        while i < len(node.keys) and key > node.keys[i]:
            i += 1

        if i < len(node.keys) and key == node.keys[i]:
            return node, i  # คืนค่า node และ index ของ key

        if node.leaf:
            return None, None

        return self._search_for_update(node.children[i], key)

    def display_all_in_order(self):
        """แสดงข้อมูลทั้งหมดใน B-Tree โดยเรียงตาม key"""
        def _in_order_traversal(node):
            if node:
                # เยี่ยมชมลูกซ้ายสุด
                for i in range(len(node.keys)):
                    if not node.leaf:
                        _in_order_traversal(node.children[i])
                    print(f"Key: {node.keys[i]}, Data: {node.data[i]}")
                if not node.leaf:
                    _in_order_traversal(node.children[len(node.keys)])
        
        print("All keys and data in order:")
        _in_order_traversal(self.root)


4. ให้นักศึกษาเพิ่มเมธอดสำหรับค้นหาข้อมูลแบบช่วง (range search)

In [14]:
class BTree:
    def __init__(self, order):
        self.root = None
        self.order = order
        self.min_keys = (order - 1) // 2
        self.max_keys = order - 1

    def insert(self, key, data):
        if self.root is None:
            self.root = BTreeNode(min_keys=self.min_keys, max_keys=self.max_keys)
            self.root.keys.append(key)
            self.root.data.append(data)
            return

        if len(self.root.keys) == self.max_keys:
            new_root = BTreeNode(leaf=False, min_keys=self.min_keys, max_keys=self.max_keys)
            new_root.children.append(self.root)
            self._split_child(new_root, 0)
            self.root = new_root

        self._insert_non_full(self.root, key, data)

    def _insert_non_full(self, node, key, data):
        i = len(node.keys) - 1
        if node.leaf:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1
            node.keys.insert(i, key)
            node.data.insert(i, data)
        else:
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1

            if len(node.children[i].keys) == self.max_keys:
                self._split_child(node, i)
                if key > node.keys[i]:
                    i += 1

            self._insert_non_full(node.children[i], key, data)

    def _split_child(self, parent, i):
        order = self.order
        child = parent.children[i]
        new_node = BTreeNode(child.leaf, min_keys=self.min_keys, max_keys=self.max_keys)
        mid = (order - 1) // 2

        parent.keys.insert(i, child.keys[mid])
        parent.data.insert(i, child.data[mid])

        new_node.keys = child.keys[mid + 1:]
        new_node.data = child.data[mid + 1:]
        if not child.leaf:
            new_node.children = child.children[mid + 1:]

        child.keys = child.keys[:mid]
        child.data = child.data[:mid]
        if not child.leaf:
            child.children = child.children[:mid + 1]

        parent.children.insert(i + 1, new_node)

    def display(self):
        """แสดงโครงสร้างของ B-Tree"""
        def _display(node, level):
            if node:
                # แสดง key และ data ในโหนดปัจจุบัน
                print('  ' * level + f"Keys: {node.keys}")
                print('  ' * level + f"Data: {node.data}")
                print('  ' * level + f"Is Leaf: {node.leaf}")
                print('  ' * level + f"Number of children: {len(node.children)}")
                print()
                # แสดงลูกทุกตัวแบบ recursive
                for child in node.children:
                    _display(child, level + 1)
        
        print("B-Tree Structure:")
        _display(self.root, 0)

    def delete(self, key):
        """ลบข้อมูลจาก B-Tree"""
        if self.root is None:
            print("The tree is empty!")
            return

        self._delete(self.root, key)

        # ถ้า root มีแค่ 1 ลูกหลังจากการลบ, ทำให้ root เป็นลูก
        if len(self.root.keys) == 0:
            if not self.root.leaf:
                self.root = self.root.children[0]
            else:
                self.root = None

    def _delete(self, node, key):
        """ฟังก์ชันลบในโหนด"""
        i = 0
        while i < len(node.keys) and key > node.keys[i]:
            i += 1

        if i < len(node.keys) and key == node.keys[i]:
            # ถ้า key อยู่ในโหนดใบ
            if node.leaf:
                node.keys.pop(i)
                node.data.pop(i)
            else:
                # ถ้า key อยู่ในโหนดที่ไม่ใช่ใบ
                self._delete_internal_node(node, i)
        elif not node.leaf:
            self._delete_in_non_leaf(node, i, key)

    def _delete_internal_node(self, node, i):
        """จัดการการลบจากโหนดที่ไม่ใช่ใบ"""
        key = node.keys[i]
        left_child = node.children[i]
        right_child = node.children[i + 1]

        if len(left_child.keys) > self.min_keys:
            predecessor = self._get_predecessor(left_child)
            node.keys[i] = predecessor
            node.data[i] = predecessor[1]
            self._delete(left_child, predecessor[0])
        elif len(right_child.keys) > self.min_keys:
            successor = self._get_successor(right_child)
            node.keys[i] = successor
            node.data[i] = successor[1]
            self._delete(right_child, successor[0])
        else:
            self._merge_children(node, i)
            self._delete(left_child, key)

    def _get_predecessor(self, node):
        """ค้นหาค่าที่มากที่สุดในลูกซ้าย"""
        while not node.leaf:
            node = node.children[-1]
        return node.keys[-1], node.data[-1]

    def _get_successor(self, node):
        """ค้นหาค่าที่น้อยที่สุดในลูกขวา"""
        while not node.leaf:
            node = node.children[0]
        return node.keys[0], node.data[0]

    def _merge_children(self, node, i):
        """รวมลูกทั้งสองให้เป็นหนึ่ง"""
        left_child = node.children[i]
        right_child = node.children[i + 1]

        left_child.keys.append(node.keys[i])
        left_child.data.append(node.data[i])

        left_child.keys.extend(right_child.keys)
        left_child.data.extend(right_child.data)

        if not left_child.leaf:
            left_child.children.extend(right_child.children)

        node.keys.pop(i)
        node.data.pop(i)
        node.children.pop(i + 1)

    def _delete_in_non_leaf(self, node, i, key):
        """จัดการการลบในลูกที่ไม่ใช่ใบ"""
        child = node.children[i]

        if len(child.keys) > self.min_keys:
            self._delete(child, key)
        else:
            left_sibling = None
            right_sibling = None

            if i > 0:
                left_sibling = node.children[i - 1]
            if i < len(node.children) - 1:
                right_sibling = node.children[i + 1]

            if left_sibling and len(left_sibling.keys) > self.min_keys:
                self._borrow_from_left(node, i)
            elif right_sibling and len(right_sibling.keys) > self.min_keys:
                self._borrow_from_right(node, i)
            else:
                if left_sibling:
                    self._merge_children(node, i - 1)
                else:
                    self._merge_children(node, i)

                self._delete(child, key)

    def _borrow_from_left(self, node, i):
        """ยืมคีย์จากลูกซ้าย"""
        child = node.children[i]
        left_sibling = node.children[i - 1]

        child.keys.insert(0, node.keys[i - 1])
        child.data.insert(0, node.data[i - 1])

        node.keys[i - 1] = left_sibling.keys.pop()
        node.data[i - 1] = left_sibling.data.pop()

        if not left_sibling.leaf:
            child.children.insert(0, left_sibling.children.pop())

    def _borrow_from_right(self, node, i):
        """ยืมคีย์จากลูกขวา"""
        child = node.children[i]
        right_sibling = node.children[i + 1]

        child.keys.append(node.keys[i])
        child.data.append(node.data[i])

        node.keys[i] = right_sibling.keys.pop(0)
        node.data[i] = right_sibling.data.pop(0)

        if not right_sibling.leaf:
            child.children.append(right_sibling.children.pop(0))

    def update(self, key, new_data):
        """อัปเดตข้อมูลใน B-Tree"""
        if self.root is None:
            print("The tree is empty!")
            return

        node, index = self._search_for_update(self.root, key)

        if node is None:
            print(f"Key {key} not found in the tree!")
            return
        
        # อัปเดตข้อมูลที่พบ
        node.data[index] = new_data
        print(f"Updated key {key} with new data: {new_data}")

    def _search_for_update(self, node, key):
        """ค้นหาคีย์ที่ต้องการอัปเดต"""
        i = 0
        while i < len(node.keys) and key > node.keys[i]:
            i += 1

        if i < len(node.keys) and key == node.keys[i]:
            return node, i  # คืนค่า node และ index ของ key

        if node.leaf:
            return None, None

        return self._search_for_update(node.children[i], key)

    def range_search(self, low_key, high_key):
        """ค้นหาข้อมูลในช่วงที่กำหนด"""
        results = []

        def _range_search_node(node):
            if node:
                # ตรวจสอบ key และเพิ่มข้อมูลในช่วงที่กำหนด
                for i in range(len(node.keys)):
                    if low_key <= node.keys[i] <= high_key:
                        results.append((node.keys[i], node.data[i]))
                    if node.keys[i] > high_key:
                        break
                
                # ถ้ามีลูกซ้ายหรือขวาที่ตรงกับช่วงก็เยี่ยมชม
                if not node.leaf:
                    for child in node.children:
                        _range_search_node(child)
        
        _range_search_node(self.root)
        return results
